# LLM agents with memory in neo4j


## Import libaries and prepare the API credential

In [10]:
pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install pyautogen

Note: you may need to restart the kernel to use updated packages.


In [12]:

from neo4j import GraphDatabase

from autogen import ConversableAgent, register_function, GroupChatManager, GroupChat
from autogen.coding import LocalCommandLineCodeExecutor

with open('openai.credential', 'r') as file:
    key = file.read()
MODEL = 'gpt-4o'

## A nested tool agents to retrieve paper info

### Python functions 

A set of functions to get citations and abstracts of a paper, search for a paper from its title, and covert between DOIs and PubMed IDs.


In [13]:
from neo4j import GraphDatabase

gdb = GraphDatabase.driver('neo4j://neo4j', auth = ("neo4j", "llm-agents"))
gdb.verify_connectivity()

def query_neo4j(query:str)->str:

    try:
        records, summary, keys = gdb.execute_query(query)
        return repr(records)
    except Exception as e:
        return repr(e) 
    



### Tool executor (just to wrap up the function)

In [14]:

graph_tool_executor = ConversableAgent(
    "graph_tool_executor",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config=False,
    is_termination_msg=lambda msg: (msg["content"]) and ("terminate" in msg["content"].lower())
)


### Tool driver (caller of the function)

In [15]:
graph_tool_driver = ConversableAgent(
    "graph_tool_driver",
    system_message = "You are curious about person, and you can use a tool (linked to a graph database) to store the information about the persons you heard. Once you get a description about a person, find the person in the graph database as a node (or create one node if the this person does not exist in the database), and save the description for this node. After that, please identify all the other persons mentiond in this description, save them into the graph as nodes, together with the relation between the original person and the other person. Finally, find one person from the graph who does not have a description yet, and say 'I want to know more about <the name of this person>. TERMINATE'. Try not to query the database too often",
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    code_execution_config=False
)

### Register the tool to both caller and executor


In [16]:
register_function(
    query_neo4j,
    caller = graph_tool_driver,
    executor = graph_tool_executor,
    description = "Query or modify the neo4j graph database. The input is a cypher query, and the output is a list of records returned from the query. We assume that the graph has nodes with label 'Person', which has 'name' and 'description' as attribute. A person may have 'Relation' with other person, and the relation has a property 'name'. Remember that 'Person' is the only allowed label of nodes. Do not introduce or use other labels."
)




### Make the nested agent for tools. 
Now lib_tool_executor is the only one exposed to the group

In [17]:
def second_last_msg(sender: ConversableAgent, recipient: ConversableAgent, summary_args: dict):
    return sender.chat_messages[recipient][-2]["content"]

nested_chats = [
    {
        "recipient": graph_tool_driver,
        "max_turns": 6,
        "summary_method": "last_msg"
    }
]

graph_tool_executor.register_nested_chats(
    nested_chats, 
    trigger = lambda sender: sender not in [graph_tool_driver]
)

In [18]:
person_expert = ConversableAgent(
    "person_expert",
    system_message = "You are a helpful assistant. Once you were asked about a person, reply a short description (<100 words) about this person, including his/her relation with some other persons in the description.",
    llm_config = {"config_list": [{"model": MODEL, "api_key": key}]},
    #is_termination_msg=lambda msg: "" in msg["content"].lower(),
    human_input_mode="ALWAYS",
)

chat_result = graph_tool_executor.initiate_chat(
    person_expert,
    message="I want to know something about Zhou Shuren",
    max_turns=10
)

graph_tool_executor (to person_expert):

I want to know something about Zhou Shuren

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
person_expert (to graph_tool_executor):

Zhou Shuren, better known by his pen name Lu Xun, was a leading Chinese writer, critic, and essayist of the early 20th century. Born in 1881 in Shaoxing, Zhejiang Province, he is considered one of the most influential figures in modern Chinese literature. Lu Xun was associated with the New Culture Movement and played a key role in the promotion of the Vernacular Chinese literary style. His works, often filled with critical observations about Chinese society and culture, include famous short stories like "A Madman's Diary" and "The True Story of Ah Q." He was also a mentor and inspiration to later generations of Chinese writers.

--------------------------------------------------------------------------------

>>>>>>>> 

KeyboardInterrupt: Interrupted by user